<a href="https://colab.research.google.com/github/inyminymoe/Data-Analysis-with-Open-Source/blob/main/%EC%98%A4%ED%94%88%EC%86%8C%EC%8A%A4%EA%B8%B0%EB%B0%98%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D(%EC%A4%91%EA%B0%84%EA%B3%BC%EC%A0%9C%EB%AC%BC)_%EA%B0%95%EC%9D%B8%EC%98%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

API 호출

In [2]:
# 설치 및 import
!pip install pandas requests

import requests
import pandas as pd

In [3]:
# API 설정
api_key = '7657646572696e7931313561474d6b41'
year, month = 2024, 12
url = 'http://openapi.seoul.go.kr:8088/7657646572696e7931313561474d6b41/json/energyUseDataStatInfo/1/5/2024/12/'

# API 호출
response = requests.get(url)

# 데이터 출력
print('URL:', url )
print('status_code:', response.status_code)
print('Content-Type:', response.headers.get('Content-Type'))

# 데이터(JSON) 확인
data = response.json()
print('data:', data)
rows = data['energyUseDataStatInfo']['row']

# 호출 성공/실패 출력
try:
    response.raise_for_status()
    print('API 호출 성공')

except requests.exceptions.HTTPError as e:
    print('API 호출 실패:', e)

df = pd.DataFrame(rows)
print(df.head())

URL: http://openapi.seoul.go.kr:8088/7657646572696e7931313561474d6b41/json/energyUseDataStatInfo/1/5/2024/12/
status_code: 200
Content-Type: application/json;charset=UTF-8
data: {'energyUseDataStatInfo': {'list_total_count': 4110, 'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}, 'row': [{'YEAR': '2024', 'MON': '12', 'HOSTCODE': '강남구', 'DONGCODE': '기타', 'MM_TYPE': '학교', 'TYPE_DIV': '단체', 'TYPE_DIV_CD': 'TB_GROUP', 'CNT': '1', 'EUS': '40385', 'EUS1': '38786', 'EUS2': '38383', 'ECO2_1_MATH_YEAR': '1599', 'ECO2_2_MATH_YEAR': '677.976', 'ECO2_1_MATH': '1800.5', 'ECO2_2_MATH': '763.412', 'ECO2_1': '1800.5', 'ECO2_2': '1800.5', 'E_TOE_YEAR': '0.36777', 'E_RATE_YEAR': '4.123', 'E_TOE_AVG': '0.414', 'E_RATE_AVG': '4.666', 'GUS': '3471', 'GUS1': '6841', 'GUS2': '5855', 'GCO2_1_MATH_YEAR': '-3370', 'GCO2_2_MATH_YEAR': '-7548.8', 'GCO2_1_MATH': '-2877', 'GCO2_2_MATH': '-6444.48', 'GCO2_1': '-2877', 'GCO2_2': '-5754', 'G_TOE_YEAR': '-3.515', 'G_RATE_YEAR': '-49.262', 'G_TOE_AVG': '-3.001', 

In [7]:
# API 및 기간 설정
api_key = '7657646572696e7931313561474d6b41'
endpoint = 'energyUseDataSummaryInfo'
YEARS = range(2015, 2025)
start, end = 1, 1000
data_list = []

def build_url(y:int, m:int,) -> str:
    return f"http://openapi.seoul.go.kr:8088/{API_KEY}/json/{endpoint}/{start}/{end}/{year}/{month:02d}"

def is_personal(row):
    if str(row.get("MM_TYPE","")).strip() == "개인":
        return True
    cd = str(row.get("MM_TYPE", "")).upper()
    return("PERSON" in cd) or ("TB_PERSON" in cd)

# 개인 + 현년만 모으기
all_rows = []
for y in YEARS:
    for m in range(1, 12+1):
        requests = requests.get(url(y, m), timeout=30)
        if requests.status_code != 200:
            print(f'[{y}-{m:02d}] HTTP {requests.status_code}')
            continue
        try:
            rows = r.json().get(endpoint, {}).get('row', []) or []
        except Exception:
            print(f'[{y}-{m:02d}] JSON 파싱 실패')
            continue
        personal = [row for row in rows if is_personal(row)]

        for row in personal:
            row['year'] = y
            row['month'] = m
        print(f'[{y}-{m:02d}] 개인 {len(personal)}건')
        all_rows.extend(personal)

df = pd.DataFrame(all_rows)
print("\n=== (2-1) DataFrame info ===")
df.info()
print("\n=== (2-1) Head ===")
display(df.head())


TypeError: 'str' object is not callable

In [9]:
import requests
import pandas as pd

API_KEY = '7657646572696e7931313561474d6b41'
ENDPOINT = 'energyUseDataSummaryInfo'
YEARS = range(2015, 2025)
START, END = 1, 1000


def url_of(y, m):
    return f"http://openapi.seoul.go.kr:8088/{API_KEY}/json/{ENDPOINT}/{START}/{END}/{y}/{m:02d}"

all_rows = []
for y in YEARS:
    for m in range(1, 13):
        try:
            r = requests.get(url_of(y, m), timeout=30)
            if r.status_code != 200:
                print(f"[{y}-{m:02d}] HTTP {r.status_code}")
                continue
            js = r.json()
            rows = js.get(ENDPOINT, {}).get("row", []) or []
            # 메타 추가
            for row in rows:
                row["year"] = y
                row["month"] = m
            all_rows.extend(rows)
        except Exception as e:
            print(f"[{y}-{m:02d}] 스킵: {type(e).__name__}")

df_raw = pd.DataFrame(all_rows)
print("전체 행수:", len(df_raw))
display(df_raw.head())

print("=== 실제 컬럼 목록 ===")
print(sorted(df_raw.columns.tolist()))

for col in ["MM_TYPE"]:
    if col in df_raw.columns:
        print(f"\n=== {col} 유니크 값(상위 30개) ===")
        vc = df_raw[col].value_counts(dropna=False)
        display(vc.head(30))
    else:
        print(f"\n=== {col} 컬럼 없음 ===")


df = df_raw.copy()

# YEAR/MON 숫자화
df["year"]  = pd.to_numeric(df["YEAR"], errors="coerce").astype("Int64")
df["month"] = pd.to_numeric(df["MON"],  errors="coerce").astype("Int64")

# 개인만 추출
df_person = df[df["MM_TYPE"] == "개인"].copy()
print("개인 행 수:", len(df_person))
display(df_person.head())


전체 행수: 840


,YEAR,MON,MM_TYPE,CNT,EUS,EUS1,EUS2,ECO2_1,ECO2_2,GUS,...,WCO2_1,WCO2_2,HUS,HUS1,HUS2,HCO2_1,HCO2_2,REG_DATE,year,month
0,2015,01,개인,767791,193784708,194781915,204969429,-6090964,-2582568.736,59133720,...,8179.308,2715.530256,22740838.937,23400055.303,27090493.875,-2504435.652,-33660084.213069,2015-06-04 17:03:55.0,2015,1
1,2015,01,학교,1382,134955565,128707423,145561511,-2178902,-923854.448,9107197,...,-39498.1,-13113.3692,0,0,0,0,0,2015-06-04 17:03:55.0,2015,1
2,2015,01,종교단체,32,372270,363376,411363,-15099.5,-6402.188,53331,...,1411.5,468.618,0,0,0,0,0,2015-06-04 17:03:55.0,2015,1
3,2015,01,소상공인,2058,10249618,9927610,11039593,-233983.5,-99209.004,536119,...,369.9,122.8068,0,0,0,0,0,2015-06-04 17:03:55.0,2015,1
4,2015,01,공동주택관리소,1747,327102096,329852334,351694752,-13671447,-5796693.528,22834274,...,-26645.4,-8846.2728,0,0,0,0,0,2015-06-04 17:03:55.0,2015,1


=== 실제 컬럼 목록 ===
['CNT', 'ECO2_1', 'ECO2_2', 'EUS', 'EUS1', 'EUS2', 'GCO2_1', 'GCO2_2', 'GUS', 'GUS1', 'GUS2', 'HCO2_1', 'HCO2_2', 'HUS', 'HUS1', 'HUS2', 'MM_TYPE', 'MON', 'REG_DATE', 'WCO2_1', 'WCO2_2', 'WUS', 'WUS1', 'WUS2', 'YEAR', 'month', 'year']

=== MM_TYPE 유니크 값(상위 30개) ===


,count
MM_TYPE,
개인,120
학교,120
종교단체,120
소상공인,120
공동주택관리소,120
기업,120
공공기관,120


개인 행 수: 120


,YEAR,MON,MM_TYPE,CNT,EUS,EUS1,EUS2,ECO2_1,ECO2_2,GUS,...,WCO2_1,WCO2_2,HUS,HUS1,HUS2,HCO2_1,HCO2_2,REG_DATE,year,month
0,2015,01,개인,767791,193784708,194781915,204969429,-6090964,-2582568.736,59133720,...,8179.308,2715.530256,22740838.937,23400055.303,27090493.875,-2504435.652,-33660084.213069,2015-06-04 17:03:55.0,2015,1
7,2015,02,개인,774620,189974230,193611430,200055533,-6859251.5,-2908322.636,56487358,...,-173889.9085,-57731.449622,18793320.213,19032221.757,21414367.325,-1429974.328,-17311712.76832,2015-06-04 16:59:30.0,2015,2
19,2015,03,개인,783726,175215719,177160755,182354932,-5541536.5,-2349611.476,49984873,...,-3588.865,-1191.50318,13826348.508,12628762.116,14509117.099,257408.9005,11088622.213667,2015-06-12 04:00:01.0,2015,3
27,2015,04,개인,798971,183891769,187268491,193988954,-6736953.5,-2856468.284,37306841,...,-130226.701,-43235.264732,5836002.885,5534118.758,8275382.274,-1068747.631,-27674867.802062,2015-07-05 04:00:00.0,2015,4
28,2015,05,개인,810422,177289769,181972266,188230004,-7811366,-3312019.184,22060892,...,-113657.49,-37734.28668,1320895.801,1558697.648,1709192.194,-313049.12,-5190153.166471,2015-08-05 04:00:01.0,2015,5


In [10]:
def season(m):
    if 3 <= m <= 5:  return "봄"
    if 6 <= m <= 8:  return "여름"
    if 9 <= m <= 11: return "가을"
    return "겨울"  # 12,1,2

df_person["season"] = df_person["month"].apply(lambda x: season(int(x)) if pd.notna(x) else None)

print("=== year / month / season 확인 ===")
display(df_person[["year","month","season"]].head())

=== year / month / season 확인 ===


,year,month,season
0,2015,1,겨울
7,2015,2,겨울
19,2015,3,봄
27,2015,4,봄
28,2015,5,봄


In [11]:
for col in ["EUS","GUS","WUS","HUS"]:
    df_person[col] = pd.to_numeric(df_person[col], errors="coerce").fillna(0.0)

df_person["total_usage"] = df_person[["EUS","GUS","WUS","HUS"]].sum(axis=1)

# 체크: 연도별 총합(3-1용 데이터)
yearly_total = df_person.groupby("year")[["EUS","GUS","WUS","HUS","total_usage"]].sum().sort_index()
print("연도별 합계(개인):")
display(yearly_total)

# 체크: 계절별 가스 평균(3-2용 데이터)
season_gas = df_person.groupby("season")["GUS"].mean()
print("계절별 가스 평균(개인):")
display(season_gas)


연도별 합계(개인):


,EUS,GUS,WUS,HUS,total_usage
year,,,,,
2015,2307639393,3.300242e+08,1.623915e+08,9.976170e+07,2.899817e+09
2016,2624619777,3.692934e+08,1.639438e+08,1.020500e+08,3.259907e+09
2017,2725339685,4.089969e+08,1.629142e+08,1.120190e+08,3.409270e+09
2018,2943309193,4.591039e+08,1.633478e+08,1.164322e+08,3.682193e+09
2019,2983329866,4.430955e+08,1.688443e+08,1.954659e+08,3.790736e+09
2020,3147833086,4.417741e+08,1.775491e+08,1.390113e+08,3.906168e+09
2021,3357544310,4.445310e+08,1.799091e+08,1.231913e+08,4.105176e+09
2022,3353529606,4.628330e+08,1.762838e+08,1.399096e+08,4.132556e+09
2023,3400871711,4.153717e+08,1.740865e+08,1.304122e+08,4.120742e+09


계절별 가스 평균(개인):


,GUS
season,
가을,1.501570e+07
겨울,6.657663e+07
봄,4.443600e+07
여름,1.127243e+07
